In [ ]:
import os
import pandas as pd
import pydicom
import h5py
import numpy as np
from scipy import ndimage
from matplotlib import pyplot, cm
from matplotlib import pyplot as plt
import cv2
import matplotlib.image as mpimg
import math
import seaborn as sns

In [ ]:
PATH_IMAGE_INFO = 'D:\\file_paths_tof.csv'
data_info = pd.read_csv(PATH_IMAGE_INFO)
names = ['pat_id','path', 'sequence', 'thickness']
data_info.columns = names
data_info['pat_id'] = data_info['pat_id'].astype(int)
data_info.head()

In [ ]:
def image_to_array(pat_id):
    file_list = (data_info.loc[data_info['pat_id'] == pat_id, 'path']).tolist()
    ref_file = pydicom.read_file(file_list[0]) 
    image_dims_3d = (int(ref_file.Rows), int(ref_file.Columns), len(file_list))
    origArray = np.zeros(image_dims_3d, dtype=ref_file.pixel_array.dtype)
    for filename in file_list:
        ds = pydicom.read_file(filename)
        origArray[:, :, file_list.index(filename)] = ds.pixel_array
        
    return origArray

In [ ]:
def normalize_array(array):
    min = np.min(array)
    max = np.max(array)
    normalized = (array - min) / (max - min)
    return normalized

In [ ]:
def scale_array_3D(array, dims):
    ## scales array to IMAGE_DIMENSIONS_3D and normalizes to [0,1]
    
    scaling_factor = [dims[0]/array.shape[0], dims[1]/array.shape[1], dims[2]/array.shape[2]]
    scaledArray = ndimage.zoom(array, scaling_factor)
    scaledArray = normalize_array(scaledArray)

    print("Scaled image size: {}".format(dims))
    return scaledArray

In [ ]:
IMAGE_DIMENSIONS_TOF = (640, 560, 200)

In [ ]:
def plot_image_hist(pid):
    image = image_to_array(pid)
    image = scale_array_3D(image, IMAGE_DIMENSIONS_TOF)
    image_count = image.shape[2]
    fig, axes = plt.subplots(image_count,2, gridspec_kw={'width_ratios': [2, 1.5]}, figsize = (10, 3*image_count))
    plt.suptitle("Patient {}".format(pid), fontsize = 20)
    for i in range(image_count):
        values = image[:,:,i].flatten()
        slice = image[:,:,i]
        sns.distplot(ax = axes[i, 0], a = values, bins = 60)
        ax = fig.add_subplot(image_count, 2, (i+1)*2)
        ax.set_aspect('equal', 'box')
        pos = ax.imshow(slice, cmap='gray', vmin = 0, vmax = 1)
        cbar = fig.colorbar(pos, ax=axes[i,1])
        axes[i, 0].set(xlim=(0, 1))
        axes[i, 1].set_axis_off()

    cbar.minorticks_on()
    p = 'C:/Users/Schule/OneDrive/Studium ZHAW/Bachelorarbeit/Deskriptive Analyse der Bilder/Histogramm_Serie_Pat_{}.png'.format(pid)
    fig.savefig(p)

In [ ]:
plot_image_hist(1)

In [ ]:
plot_image_hist(292)

In [ ]:
image = image_to_array(292)
image = scale_array_3D(image, IMAGE_DIMENSIONS_TOF)
print(np.min(image), np.max(image))